In [2]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 2 Preprocessing :
* Delete Duplicate
* Impute Null pake iterative imputer
* Outlier capping di sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [3]:
df = pd.read_csv('../../Punya Andi/UFC_kombinasi2.csv')

In [4]:
df.head()

,weight_class,B_avg_opp_SIG_STR_att,B_avg_TOTAL_STR_att,B_avg_opp_TOTAL_STR_att,B_avg_opp_HEAD_att,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_DISTANCE_att,B_win_by_KO/TKO,B_Height_cms,...,R_avg_opp_TOTAL_STR_att,R_avg_BODY_att,R_avg_opp_BODY_att,R_avg_DISTANCE_att,R_win_by_KO/TKO,R_Height_cms,R_Reach_cms,R_Weight_lbs,gender,B_Reach_cms
0,7.0,82.500000,37.000000,131.687500,65.000000,11.000000,6.937500,71.000000,0.0,182.88,...,97.000000,8.250000,5.000000,40.000000,0.0,182.88,177.80,185.0,1.0,190.50
1,5.0,89.953125,141.210938,91.882812,83.210938,3.960938,2.070312,88.695312,3.0,177.80,...,129.593750,20.125000,17.593750,98.250000,0.0,180.34,187.96,170.0,1.0,177.80
2,6.0,74.937500,66.000000,97.914062,59.750000,8.000000,5.218750,64.500000,0.0,182.88,...,113.221941,12.500000,2.968750,76.750000,0.0,185.42,185.42,170.0,1.0,190.50
3,6.0,37.226562,51.500000,52.046875,28.773438,6.656250,4.453125,24.234375,3.0,175.26,...,125.441042,8.181122,8.554874,67.441225,4.0,177.80,185.42,170.0,1.0,175.26
4,3.0,93.299805,102.170410,125.859863,76.181396,10.418213,9.236328,69.770264,0.0,165.10,...,165.000000,20.125000,19.159180,88.000000,3.0,160.02,167.64,135.0,1.0,170.18


In [5]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

# Create the KNN model
knn = KNeighborsRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2')

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)


Best Parameters: {'leaf_size': 10, 'metric': 'manhattan', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}
Best R-squared Score: 0.7292596941270275


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Evaluate the best model using 5-fold cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='r2')

# Calculate the mean R-squared
mean_r2 = cv_scores.mean()
print("Mean R-squared:", mean_r2)

Mean R-squared: 0.7292596941270275


In [9]:
# Calculate the RMSE using 5-fold cross-validation
cv_rmse = np.sqrt(np.abs(cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')))

# Calculate the mean RMSE
mean_rmse = cv_rmse.mean()
print("Mean RMSE:", mean_rmse)


Mean RMSE: 4.881872371825598


### Kaggle Prediction

In [13]:
df_test = pd.read_csv('../Kaggle_Test_Data.csv')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1)
df_test = df_test.reindex(columns=df.columns, fill_value=0)
df_test = df_test.drop(['B_Reach_cms'], axis=1)
df_test = scaler.transform(df_test)
kaggle_pred = grid_search.best_estimator_.predict(df_test)
kaggle_pred = pd.DataFrame(kaggle_pred)
kaggle_pred = pd.concat([df_test_id, kaggle_pred], axis=1)
pd.DataFrame(kaggle_pred).to_csv('./kaggle/kombinasi1_knn', index=False)

ValueError: Input X contains NaN.
KNeighborsRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values